# Get test data from checklist test

As shown in the end of the SQuAD-create-test-suite.ipynb, we can get all examples from the test suite. However, I'm not sure how to get the correct answers, especially for the invariance test cases. So I'm going to process and save all the test examples in csv files and read in later. 


This file is copied from SQuAD-create-test-suite.ipynb. I removed the irrelavant cells. 


In [1]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser
from transformers import pipeline 

model_name = "trained_model_squad1/"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = pipeline('question-answering', model=model, tokenizer=tokenizer)


In [14]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb
import datasets
import pandas as pd
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
# SAVE SQUAD DATASET TO CSV 

dataset = datasets.load_dataset('squad')
# def format_dataset(example):
#     """
#     format answers from dict to json
#     so that data looks consistent when exporting to csv
#     """
    example['answers'] = json.dumps(example['answers'])
#     return example

# dataset = dataset.map(format_dataset)
# dataset['train'].to_csv('train.csv', index=None)
df_train = pd.read_csv('train.csv')
df_train.head(1)

Reusing dataset squad (/home/syang/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/87599 [00:00<?, ?ex/s]

  0%|          | 0/10570 [00:00<?, ?ex/s]

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{""text"": [""Saint Bernadette Soubirous""], ""answ..."


In [4]:
import pandas as pd
def dataset_fmt(t, file_name):
    """
    format t to acceptable dataframe format 
    """
    df = pd.DataFrame([
        {'context':i[k][0], 'question':i[k][1], 'answers_text': j[k]}
        for i, j in zip(t.data, t.labels)
        for k in range(len(i))
    ])
    df['answer_start'] = df.apply(lambda row: row['context'].index(row['answers_text']), axis=1)
    df['answers'] = df.apply(lambda row: {'text': [row['answers_text']], 'answer_start': [row['answer_start']]}, axis=1)
    df = df.drop(columns=['answers_text', 'answer_start'])
    df['answers'] = df['answers'].apply(json.dumps)
    df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(f'new_data/{file_name}.csv', index=False)
    return df


def inv_dataset_fmt(t, file_name, df_train=df_train):
    """
    use original answers for invariance cases
    note that in many cases start position changes, that's why we find the correct index 
    """
    df = pd.DataFrame([{'context':i[0][0], 'question':i[0][1], 'context_modified':i[1][0], 'question_modified':i[1][1]} for i in t.data])
    df = (
        df
        .merge(df_train, on=['context','question'], how='inner')
        .drop(columns=['context', 'question', 'id', 'title'])
        .rename(columns={'context_modified': 'context', 'question_modified':'question'})
        .drop_duplicates(subset=['context','question'])
        .reset_index(drop=True)
    )
    df = find_matching_index(df)
    df['answers'] = df['answers'].apply(json.dumps)
    df.to_csv(f'new_data/{file_name}.csv', index=False)
    return df

In [5]:
def find_all_matching(string, sentence):
    matching_lst = []
    for m in re.finditer(re.escape(string), sentence): #re.escape deals with weird character issues
        matching_lst.append(m.start())
    return matching_lst

def find_matching_index(df):
    for index, row in df.iterrows(): 
        row['answers'] = json.loads(row['answers'])
        for i in range(len(row['answers']['text'])):
            original_index = row['answers']['answer_start'][i]
            all_matched_indexes = find_all_matching(row['answers']['text'][i], row['context'])
            if len(all_matched_indexes) ==0:
                pass
            elif original_index in all_matched_indexes:
                # if the original index is in the matched indexes, do nothing
                pass
            else:
                print(index)
                print(row)
                print('changed')
                # if not, choose the index that's the closest to the original index
                row['answers']['answer_start'][i] = min(all_matched_indexes, key=lambda x: abs(x - original_index))
                print(original_index)
                print(row['answers']['answer_start'][i])
    return df

In [8]:
editor = checklist.editor.Editor()
editor.tg

## Vocabulary

In [9]:
print(', '.join(editor.suggest('{first_name} is {mask} than {first_name2}.')[:60]))

/home/syang/miniconda3/envs/nlp/lib/python3.9/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


smarter, better, older, younger, taller, worse, different, stronger, cooler, nicer, tougher, shorter, bigger, hotter, more, darker, happier, smaller, faster, richer, wiser, thinner, less, weaker, larger, quieter, cleaner, closer, healthier, heavier, colder, slower, harder, wealthier, safer, quicker, longer, higher, cheaper, thicker, louder, sharper, lighter, warmer, brighter, greater, deeper, lower, easier, softer, smoother, poorer, other, stranger, newer, stricter, simpler, clearer, superior, tighter


In [10]:
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


In [11]:
adj[2]

('tall', 'tall')

In [12]:
t = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    )
    ],
    labels = ['{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=500,
    save=True
    )
name = 'A is COMP than B. Who is more COMP?'
# description = ''
# test = MFT(**t, name=name, description=description, capability='Vocabulary')
# suite.add(test)

In [15]:
dataset_fmt(t, "compare_more")

,context,question,answers
0,Jim is cleaner than Amanda.,Who is cleaner?,"{""text"": [""Jim""], ""answer_start"": [0]}"
1,Anthony is richer than Lucy.,Who is richer?,"{""text"": [""Anthony""], ""answer_start"": [0]}"
2,Donald is smarter than Patricia.,Who is smarter?,"{""text"": [""Donald""], ""answer_start"": [0]}"
3,Francis is stronger than Jessica.,Who is stronger?,"{""text"": [""Francis""], ""answer_start"": [0]}"
4,Eleanor is cooler than Bob.,Who is cooler?,"{""text"": [""Eleanor""], ""answer_start"": [0]}"
...,...,...,...
493,Arthur is cleaner than Bobby.,Who is cleaner?,"{""text"": [""Arthur""], ""answer_start"": [0]}"
494,Joseph is wiser than Michael.,Who is wiser?,"{""text"": [""Joseph""], ""answer_start"": [0]}"
495,Dick is stronger than Grace.,Who is stronger?,"{""text"": [""Dick""], ""answer_start"": [0]}"
496,Caroline is richer than Claire.,Who is richer?,"{""text"": [""Caroline""], ""answer_start"": [0]}"


In [16]:
t = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    )
    ],
    labels = ['{first_name1}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=500,
    save=True
    )
name = 'A is COMP than B. Who is less COMP?'
# description = ''
# test = MFT(**t, name=name, description=description, capability='Vocabulary')
# suite.add(test)

In [17]:
dataset_fmt(t, "compare_less")

,context,question,answers
0,George is older than Joseph.,Who is less old?,"{""text"": [""Joseph""], ""answer_start"": [21]}"
1,Evelyn is smaller than Frances.,Who is less small?,"{""text"": [""Frances""], ""answer_start"": [23]}"
2,Donna is wiser than Ashley.,Who is less wise?,"{""text"": [""Ashley""], ""answer_start"": [20]}"
3,Caroline is shorter than Larry.,Who is less short?,"{""text"": [""Larry""], ""answer_start"": [25]}"
4,Jimmy is darker than Thomas.,Who is less dark?,"{""text"": [""Thomas""], ""answer_start"": [21]}"
...,...,...,...
489,Charles is greater than Sam.,Who is less great?,"{""text"": [""Sam""], ""answer_start"": [24]}"
490,Julia is higher than Francis.,Who is less high?,"{""text"": [""Francis""], ""answer_start"": [21]}"
491,Nicole is weaker than Matthew.,Who is less weak?,"{""text"": [""Matthew""], ""answer_start"": [22]}"
492,Kevin is cleaner than Don.,Who is less clean?,"{""text"": [""Don""], ""answer_start"": [22]}"


In [18]:
def crossproduct(t):
    # takes the output of editor.template and does the cross product of contexts and qas
    ret = []
    ret_labels = []
    for x in t.data:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    t.data = ret
    t.labels = ret_labels
    return t


In [19]:
state = editor.suggest('John is very {mask} about the project.')[:20]
print(', '.join(editor.suggest('John is {mask} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

very, pretty, extremely, also, still, quite, more, really, not, clearly, fairly, incredibly, particularly, now, understandably, rather, cautiously, surprisingly, certainly, feeling, so, especially, definitely, generally, most, highly, super, reportedly, being, obviously


In [20]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?'
# desc = ''
# test = MFT(**t, name=name, description=desc, capability='Vocabulary')
# suite.add(test)


In [21]:
dataset_fmt(t, "intensifiers_reducers")

,context,question,answers
0,Ken is incredibly curious about the project. D...,Who is most curious about the project?,"{""text"": [""Ken""], ""answer_start"": [0]}"
1,Ken is incredibly curious about the project. D...,Who is least curious about the project?,"{""text"": [""Diana""], ""answer_start"": [45]}"
2,Diana is curious about the project. Ken is inc...,Who is most curious about the project?,"{""text"": [""Ken""], ""answer_start"": [36]}"
3,Diana is curious about the project. Ken is inc...,Who is least curious about the project?,"{""text"": [""Diana""], ""answer_start"": [0]}"
4,Ken is curious about the project. Diana is sli...,Who is most curious about the project?,"{""text"": [""Ken""], ""answer_start"": [0]}"
...,...,...,...
5959,Pamela is mildly hopeful about the project. Ad...,Who is least hopeful about the project?,"{""text"": [""Pamela""], ""answer_start"": [0]}"
5960,Adam is super hopeful about the project. Pamel...,Who is most hopeful about the project?,"{""text"": [""Adam""], ""answer_start"": [0]}"
5961,Adam is super hopeful about the project. Pamel...,Who is least hopeful about the project?,"{""text"": [""Pamela""], ""answer_start"": [41]}"
5962,Pamela is mildly hopeful about the project. Ad...,Who is most hopeful about the project?,"{""text"": [""Adam""], ""answer_start"": [44]}"


## Taxonomy

### Size, chape, color, age, material

In [22]:
import munch
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))


In [23]:
print(', '.join(editor.suggest('There is {a:p.v1} {p.v2} {mask} in the room.', p=props, verbose=False)[:30]))
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']


sofa, couch, wall, carpet, chair, table, light, lamp, door, clock, mirror, desk, bed, TV, bar, television, window, box, tree, painting, curtain, fan, fridge, screen, wallpaper, piano, rug, shelf, camera, candle


In [24]:
t = crossproduct(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ), 
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ), 
            
        ]
        
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'size, shape, age, color'
desc = ''
# test = MFT(**t, name=name, description=desc, capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [25]:
dataset_fmt(t, "size_shape_age_color")

,context,question,answers
0,There is an oval brown table in the room.,What shape is the table?,"{""text"": [""oval""], ""answer_start"": [12]}"
1,There is an oval brown table in the room.,What color is the table?,"{""text"": [""brown""], ""answer_start"": [17]}"
2,There is a table in the room. The table is ova...,What shape is the table?,"{""text"": [""oval""], ""answer_start"": [43]}"
3,There is a table in the room. The table is ova...,What color is the table?,"{""text"": [""brown""], ""answer_start"": [52]}"
4,There is a big oval toy in the room.,What size is the toy?,"{""text"": [""big""], ""answer_start"": [11]}"
...,...,...,...
1635,There is a box in the room. The box is enormou...,What color is the box?,"{""text"": [""blue""], ""answer_start"": [52]}"
1636,There is an enormous black thing in the room.,What size is the thing?,"{""text"": [""enormous""], ""answer_start"": [12]}"
1637,There is an enormous black thing in the room.,What color is the thing?,"{""text"": [""black""], ""answer_start"": [21]}"
1638,There is a thing in the room. The thing is eno...,What size is the thing?,"{""text"": [""enormous""], ""answer_start"": [43]}"


### Professions vs nationalities

In [26]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

In [27]:
def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')

In [28]:
def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)

In [29]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True,
    ))
name = 'Profession vs nationality'
# test = MFT(**t, name=name, expect=expect_squad, description='',  capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [30]:
dataset_fmt(t, "profession_nationality")

,context,question,answers
0,Daniel is a Chinese reporter.,What is Daniel's job?,"{""text"": [""reporter""], ""answer_start"": [20]}"
1,Daniel is a Chinese reporter.,What is Daniel's nationality?,"{""text"": [""Chinese""], ""answer_start"": [12]}"
2,Daniel is a reporter. Daniel is Chinese.,What is Daniel's job?,"{""text"": [""reporter""], ""answer_start"": [12]}"
3,Daniel is a reporter. Daniel is Chinese.,What is Daniel's nationality?,"{""text"": [""Chinese""], ""answer_start"": [32]}"
4,Daniel is Chinese. Daniel is a reporter.,What is Daniel's job?,"{""text"": [""reporter""], ""answer_start"": [31]}"
...,...,...,...
4985,Katherine is Nigerian. Katherine is a producer.,What is Katherine's nationality?,"{""text"": [""Nigerian""], ""answer_start"": [13]}"
4986,Katherine is Nigerian and a producer.,What is Katherine's job?,"{""text"": [""producer""], ""answer_start"": [28]}"
4987,Katherine is Nigerian and a producer.,What is Katherine's nationality?,"{""text"": [""Nigerian""], ""answer_start"": [13]}"
4988,Katherine is a producer and Nigerian.,What is Katherine's job?,"{""text"": [""producer""], ""answer_start"": [15]}"


### Animal vs vehicle

In [31]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Animal vs Vehicle'
# test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test, overwrite=True)


In [32]:
dataset_fmt(t, "animal_vehicle")

,context,question,answers
0,Alice has a lizard and a minivan.,What animal does Alice have?,"{""text"": [""lizard""], ""answer_start"": [12]}"
1,Alice has a lizard and a minivan.,What vehicle does Alice have?,"{""text"": [""minivan""], ""answer_start"": [25]}"
2,Alice has a minivan and a lizard.,What animal does Alice have?,"{""text"": [""lizard""], ""answer_start"": [26]}"
3,Alice has a minivan and a lizard.,What vehicle does Alice have?,"{""text"": [""minivan""], ""answer_start"": [12]}"
4,Michelle has a bull and a van.,What animal does Michelle have?,"{""text"": [""bull""], ""answer_start"": [15]}"
...,...,...,...
1979,Elaine has a tractor and an iguana.,What vehicle does Elaine have?,"{""text"": [""tractor""], ""answer_start"": [13]}"
1980,Linda has a fish and a truck.,What animal does Linda have?,"{""text"": [""fish""], ""answer_start"": [12]}"
1981,Linda has a fish and a truck.,What vehicle does Linda have?,"{""text"": [""truck""], ""answer_start"": [23]}"
1982,Linda has a truck and a fish.,What animal does Linda have?,"{""text"": [""fish""], ""answer_start"": [24]}"


In [33]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} bought {a:animal}. {first_name2} bought {a:vehicle}.',
            '{first_name2} bought {a:vehicle}. {first_name} bought {a:animal}.',
        ],
        'qas': [
            (
                'Who bought an animal?',
                '{first_name}'
            ), 
            (
                'Who bought a vehicle?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Animal vs Vehicle v2'
# test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test, overwrite=True)

In [34]:
dataset_fmt(t, "animal_vehicle2")

,context,question,answers
0,Joan bought a bull. Jonathan bought a train.,Who bought an animal?,"{""text"": [""Joan""], ""answer_start"": [0]}"
1,Joan bought a bull. Jonathan bought a train.,Who bought a vehicle?,"{""text"": [""Jonathan""], ""answer_start"": [20]}"
2,Jonathan bought a train. Joan bought a bull.,Who bought an animal?,"{""text"": [""Joan""], ""answer_start"": [25]}"
3,Jonathan bought a train. Joan bought a bull.,Who bought a vehicle?,"{""text"": [""Jonathan""], ""answer_start"": [0]}"
4,Katherine bought a snake. Howard bought a train.,Who bought an animal?,"{""text"": [""Katherine""], ""answer_start"": [0]}"
...,...,...,...
1975,Heather bought a SUV. Ken bought a serpent.,Who bought a vehicle?,"{""text"": [""Heather""], ""answer_start"": [0]}"
1976,Christopher bought a duck. Marilyn bought a tr...,Who bought an animal?,"{""text"": [""Christopher""], ""answer_start"": [0]}"
1977,Christopher bought a duck. Marilyn bought a tr...,Who bought a vehicle?,"{""text"": [""Marilyn""], ""answer_start"": [27]}"
1978,Marilyn bought a truck. Christopher bought a d...,Who bought an animal?,"{""text"": [""Christopher""], ""answer_start"": [24]}"


In [35]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[0]}. {first_name2} is very {s2[0]}.',
            '{first_name2} is very {s2[0]}. {first_name} is very {s1[0]}.',
        ],
        'qas': [
            (
                'Who is {s1[1]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[1]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
   ))
t += crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[1]}. {first_name2} is very {s2[1]}.',
            '{first_name2} is very {s2[1]}. {first_name} is very {s1[1]}.',
        ],
        'qas': [
            (
                'Who is {s1[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[0]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
    )) 
name = 'Synonyms'
# test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [36]:
dataset_fmt(t, "synonyms")

,context,question,answers
0,Margaret is very vocal. Ed is very angry.,Who is outspoken?,"{""text"": [""Margaret""], ""answer_start"": [0]}"
1,Margaret is very vocal. Ed is very angry.,Who is furious?,"{""text"": [""Ed""], ""answer_start"": [24]}"
2,Ed is very angry. Margaret is very vocal.,Who is outspoken?,"{""text"": [""Margaret""], ""answer_start"": [18]}"
3,Ed is very angry. Margaret is very vocal.,Who is furious?,"{""text"": [""Ed""], ""answer_start"": [0]}"
4,Bruce is very spiritual. Helen is very intelli...,Who is religious?,"{""text"": [""Bruce""], ""answer_start"": [0]}"
...,...,...,...
1831,Marilyn is very modest. Bill is very joyful.,Who is humble?,"{""text"": [""Marilyn""], ""answer_start"": [0]}"
1832,Amy is very frightened. Emma is very furious.,Who is scared?,"{""text"": [""Amy""], ""answer_start"": [0]}"
1833,Amy is very frightened. Emma is very furious.,Who is angry?,"{""text"": [""Emma""], ""answer_start"": [24]}"
1834,Emma is very furious. Amy is very frightened.,Who is scared?,"{""text"": [""Amy""], ""answer_start"": [22]}"


In [37]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))#list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))

In [38]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'A is COMP than B. Who is antonym(COMP)? B'
# test = MFT(**t, name=name, description='', capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [39]:
dataset_fmt(t, "compare_antonym")

,context,question,answers
0,Roger is smarter than Tom.,Who is dumber?,"{""text"": [""Tom""], ""answer_start"": [22]}"
1,Roger is smarter than Tom.,Who is smarter?,"{""text"": [""Roger""], ""answer_start"": [0]}"
2,Tom is dumber than Roger.,Who is dumber?,"{""text"": [""Tom""], ""answer_start"": [0]}"
3,Tom is dumber than Roger.,Who is smarter?,"{""text"": [""Roger""], ""answer_start"": [19]}"
4,Nicole is smarter than Tom.,Who is dumber?,"{""text"": [""Tom""], ""answer_start"": [23]}"
...,...,...,...
1991,Ed is poorer than Bob.,Who is richer?,"{""text"": [""Bob""], ""answer_start"": [18]}"
1992,Sophie is bigger than Scott.,Who is smaller?,"{""text"": [""Scott""], ""answer_start"": [22]}"
1993,Sophie is bigger than Scott.,Who is bigger?,"{""text"": [""Sophie""], ""answer_start"": [0]}"
1994,Scott is smaller than Sophie.,Who is smaller?,"{""text"": [""Scott""], ""answer_start"": [0]}"


In [40]:
antonym_adjs = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is more {a[0]} than {first_name1}.',
            '{first_name1} is more {a[1]} than {first_name}.',
            '{first_name} is less {a[1]} than {first_name1}.',
            '{first_name1} is less {a[0]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is more {a[0]}?',
                '{first_name}',
            ),
            (
                'Who is less {a[0]}?',
                '{first_name1}',
            ),
            (
                'Who is more {a[1]}?',
                '{first_name1}',
            ),
            (
                'Who is less {a[1]}?',
                '{first_name}',
            ),
        ]
        ,
    },
    a = antonym_adjs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.'
# test = MFT(**t, name=name, description='', capability='Taxonomy')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [41]:
dataset_fmt(t, "compare_moreless_antonym")

,context,question,answers
0,Paul is more stupid than Tony.,Who is more stupid?,"{""text"": [""Paul""], ""answer_start"": [0]}"
1,Paul is more stupid than Tony.,Who is less stupid?,"{""text"": [""Tony""], ""answer_start"": [25]}"
2,Paul is more stupid than Tony.,Who is more smart?,"{""text"": [""Tony""], ""answer_start"": [25]}"
3,Paul is more stupid than Tony.,Who is less smart?,"{""text"": [""Paul""], ""answer_start"": [0]}"
4,Tony is more smart than Paul.,Who is more stupid?,"{""text"": [""Paul""], ""answer_start"": [24]}"
...,...,...,...
7995,Bill is less powerful than Alison.,Who is less powerful?,"{""text"": [""Bill""], ""answer_start"": [0]}"
7996,Alison is less powerless than Bill.,Who is more powerless?,"{""text"": [""Bill""], ""answer_start"": [30]}"
7997,Alison is less powerless than Bill.,Who is less powerless?,"{""text"": [""Alison""], ""answer_start"": [0]}"
7998,Alison is less powerless than Bill.,Who is more powerful?,"{""text"": [""Alison""], ""answer_start"": [0]}"


## Robustness

typos

In [ ]:
# original code from https://github.com/marcotcr/checklist/blob/115f123de47ab015b2c3a6baebaffb40bab80c9f/notebooks/SQuAD.ipynb

# import pickle
# data, answers =  load_squad()
# spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_squad.pkl', 'rb'))
# pairs = [(x['passage'], x['question']) for x in data]
# processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]

In [13]:
pairs = [(x['context'], x['question']) for x in dataset['train']]

In [14]:
# # source: https://github.com/marcotcr/checklist/blob/115f123de47ab015b2c3a6baebaffb40bab80c9f/notebooks/QQP.ipynb
# # all_questions = list(all_questions)
# # parsed_questions = list(nlp.pipe(all_questions))
# # spacy_map = dict([(x, y) for x, y in zip(all_questions, parsed_questions)])

# nlp = spacy.load('en_core_web_sm')
# all_questions = set() # a set of all questions and context
# for x in dataset['train']:
#     all_questions.add(x['question'])
#     all_questions.add(x['context'])


# parsed_questions = list(nlp.pipe(all_questions)) # this takes a while to run 
# spacy_map = dict([(x, y) for x, y in zip(all_questions, parsed_questions)])

import pickle
# pickle.dump(spacy_map, open('processed_squad.pkl', 'wb'))
# pickle.dump(parsed_questions, open('processed_questions.pkl', 'wb'))
spacy_map =  pickle.load(open('processed_squad.pkl', 'rb'))


In [15]:
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]

In [218]:
def question_typo(x):
    """
    x[0]: context
    x[1]: question 
    Perturb.add_typos(x[1]): add a typo to question 
    """
    return (x[0], Perturb.add_typos(x[1]))
t = Perturb.perturb(pairs, question_typo, nsamples=500)
# test = INV(**t, name='Question typo', capability='Robustness', description='')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad)
# suite.add(test, overwrite=True)

In [219]:
inv_dataset_fmt(t, 'typo')

,context,question,answers
0,"The name iPod was proposed by Vinnie Chieco, a...",Whatf ilm inspired the name of the iPod?,"{""text"": [""2001: A Space Odyssey""], ""answer_st..."
1,Numerous recordings of Chopin's works are avai...,On Chopin's 200tha nniversary critics of what ...,"{""text"": [""The New York Times""], ""answer_start..."
2,"From January 18–20, 2010 a national conference...",What does thep lan call for all rural Tibetan ...,"{""text"": [""education""], ""answer_start"": [627]}"
3,Albert Einstein proposed that the laws of phys...,"Before Einstein, speed was though to be relait...","{""text"": [""the luminiferous ether""], ""answer_s..."
4,Into the first half of the twentieth century a...,hWat is Zhongyang Shangchang?,"{""text"": [""huge department stores""], ""answer_s..."
...,...,...,...
495,This creates a linear pitch space in which oct...,The pitch halfway between C (60) and C♯ (61) i...,"{""text"": [""60.5""], ""answer_start"": [518]}"
496,The middle up segment is mainly occupied by Me...,"When did Seibu, located in Jakarta, er-open?","{""text"": [""2007""], ""answer_start"": [114]}"
497,"Aposematism, where organisms are brightly colo...",How does aposematism help a specise population?,"{""text"": [""Upon being harmed (e.g., stung) by ..."
498,Portuguese cuisine is diverse. The Portuguese ...,What are two popular oNrthern Portugal dishes?,"{""text"": [""arroz de sarrabulho (rice stewed in..."


Contractions

In [220]:
def contractions(x):
    conts = Perturb.contractions(x[1])
    return [(x[0], a) for a in conts]
t = Perturb.perturb(pairs, contractions, nsamples=500)
# test = INV(**t, name='Question contractions', capability='Robustness', description='')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad)
# suite.add(test)

In [221]:
inv_dataset_fmt(t, 'contractions')

,context,question,answers
0,According to a tradition first reported by Sul...,What's the meaning of Thorn Ey?,"{""text"": [""Thorn Island""], ""answer_start"": [124]}"
1,"By the verse Quran, 2:124, Shias believe that ...",Who knows everything that's needed to get to t...,"{""text"": [""Imam""], ""answer_start"": [499]}"
2,Jehovah's Witnesses is a millenarian restorati...,Who's in charges of directing the Jehovah Witn...,"{""text"": [""Governing Body of Jehovah's Witness..."
3,Toxocara canis (dog roundworm) eggs in dog fec...,What's the common name of the species that cau...,"{""text"": [""dog roundworm""], ""answer_start"": [16]}"
4,After a nonprofit organization has been formed...,What happens if an NPO doesn't abide by the ta...,"{""text"": [""losing its tax exempt status""], ""an..."
...,...,...,...
495,A few lines of the Paris Métro in France opera...,What's required in order for the guide rails t...,"{""text"": [""a single polarity supply""], ""answer..."
496,The new SuperSpeed bus provides a fourth trans...,What's full-duplex in SuperSpeed transfer mode?,"{""text"": [""Communication""], ""answer_start"": [4..."
497,"In the US, nutritional standards and recommend...",What's the name of the concept that has replac...,"{""text"": [""MyPlate""], ""answer_start"": [248]}"
498,Hayek's greatest intellectual debt was to Carl...,Carl Menger's work in social explanation wasn'...,"{""text"": [""Scottish Enlightenment""], ""answer_s..."


Add random sentence

In [222]:
random_sentences = set()
for x, _ in processed_pairs:
    for y in x.sents:
        random_sentences.add(y.text)
random_sentences = list(random_sentences)

In [223]:
# check
for y in spacy_map[pairs[0][0]].sents:
    print(y.text)

Architecturally, the school has a Catholic character.
Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".
Next to the Main Building is the Basilica of the Sacred Heart.
Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.
It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.
At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


In [224]:
len(random_sentences)

91957

In [225]:
def add_random_sentence(x, **kwargs):
    random_s = np.random.choice(random_sentences)
    while random_s in x[0]:
        random_s = np.random.choice(random_sentences)
    random_s = random_s.strip('.') + '. '
    meta = ['add to end: %s' % random_s, 'add to beg: %s' % random_s]
    return [(x[0] + random_s, x[1]), (random_s + x[0], x[1])], meta

def format_add(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s\n' % meta
    return ret

t = Perturb.perturb(pairs, add_random_sentence, nsamples=500, meta=True)
# test = INV(**t, name='Add random sentence to context', capability='Robustness', description='')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_add)
# suite.add(test)

In [227]:
inv_dataset_fmt(t, 'random_sentence')

,context,question,answers
0,"During the American Civil War, American cotton...",What did the abandonment of Egyptian cotton me...,"{""text"": [""bankruptcy""], ""answer_start"": [761]}"
1,The sovereign territory of Israel (according t...,What's home to 57% of the nation's population?,"{""text"": [""Israeli Coastal Plain""], ""answer_st..."
2,"The two became friends, and for many years liv...",What was the charity that Liszt and Chopin las...,"{""text"": [""the Beethoven Memorial in Bonn""], ""..."
3,"Each season premieres with the audition round,...","In the audition rounds, what do contestants wh...","{""text"": [""a golden ticket""], ""answer_start"": ..."
4,The value of the U.S. dollar declined signific...,What was the Federal Reserve designed to furnish?,"{""text"": [""an \""elastic\"" currency""], ""answer_..."
...,...,...,...
495,"Between 1949 and the 1980s, telephone communic...",What company has a 40% stake in OTE?,"{""text"": [""Deutsche Telekom""], ""answer_start"":..."
496,"Including Kerry, the Democratic primary race h...",What was DiFruscia's position?,"{""text"": [""State Representative""], ""answer_sta..."
497,Basic sanitation techniques can have a profoun...,What is a solution to help this problem?,"{""text"": [""implementation of educational progr..."
498,Polarization is predictable from an antenna's ...,What is a more complicated type of polarizatio...,"{""text"": [""quad antenna""], ""answer_start"": [121]}"


## NER

In [228]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both
            

In [229]:
def expect_same(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if not meta:
        return pred == orig_pred
    return pred == re.sub(r'\b%s\b' % re.escape(meta[0]), meta[1], orig_pred)

def format_replace(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

def format_replace_context(x, pred, conf, label=None, meta=None):
    ret = format_squad_with_context(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

In [230]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_names), nsamples=500, meta=True)

# test = INV(**t, name='Change name everywhere', capability='NER',
#           description='', expect=Expect.pairwise(expect_same))
# test.run(predconfs, n=100)
# test.summary(3, format_example_fn=format_replace)
# suite.add(test, overwrite=True)

In [231]:
inv_dataset_fmt(t, 'name_change')

,context,question,answers
0,"The Battle of Osan, the first significant Amer...",At what Battle did the 24th Infantry Division ...,"{""text"": [""Battle of Taejon""], ""answer_start"":..."
1,It seems to have been St Bernard of Clairvaux ...,What did the query starter believe to be the u...,"{""text"": [""How can there be absence of sin whe..."
2,"During the presidential referendum in Egypt, K...",What was Qutb's sentence?,"{""text"": [""executed""], ""answer_start"": [505]}"
3,Many of the city's buildings are in the Georgi...,What has the old Psychiatric Hospital been tur...,"{""text"": [""Atkins Hall""], ""answer_start"": [478]}"
4,Following the start of the Arab Spring in 2011...,About what percentage of the Libyan population...,"{""text"": [""30""], ""answer_start"": [810]}"
...,...,...,...
495,"This eventually led to LBJ's Civil Rights Act,...","Outside of the private business regulations, w...","{""text"": [""integrate public facilities""], ""ans..."
496,"In addition, the city has about 160 museums—th...",Where is the house that Leon Trotsky was murde...,"{""text"": [""Coyoac\u00e1n""], ""answer_start"": [6..."
497,"In 1975, a group of Chicago Cubs fans based in...",Who currently heads the Emil Verban Society?,"{""text"": [""Illinois Senator Dick Durbin""], ""an..."
498,Captain John Charles Jackson and Thomas Gilber...,At what island did the Globe arrive in 1824?,"{""text"": [""Mulgrave""], ""answer_start"": [557]}"


In [232]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_location), nsamples=500, meta=True)

# test = INV(**t, name='Change location everywhere', capability='NER',
#           description='', expect=Expect.pairwise(expect_same))
# test.run(predconfs, n=100)
# test.summary(3, format_example_fn=format_replace)
# suite.add(test, overwrite=True)

In [233]:
inv_dataset_fmt(t, 'location_change')

,context,question,answers
0,A second Irvine gastronomical claim to fame is...,Who is the founder of Louis' Lunch in Irvine?,"{""text"": [""Louis Lassen""], ""answer_start"": [216]}"
1,"In July 2013, there were 41,000 non-Germans by...",Which towns have the highest immigrant populat...,"{""text"": [""Erfurt, Jena, Weimar and Ilmenau""],..."
2,Indonesia was the last Western republic to gra...,Who was the first woman to serve on the Federa...,"{""text"": [""Elisabeth Kopp""], ""answer_start"": [..."
3,Starting in the 1890s and stretching in some p...,In what year was Alaska's capital officially c...,"{""text"": [""1906""], ""answer_start"": [295]}"
4,"John, the youngest of five sons of King Henry ...",When did Richard I become king?,"{""text"": [""1189""], ""answer_start"": [457]}"
...,...,...,...
495,The changes brought about by these development...,Historians from what country in particular do ...,"{""text"": [""Italy""], ""answer_start"": [446]}"
496,"Throughout their history, Slavs came into cont...",Who vanished from the population of the Balkans?,"{""text"": [""The Thracians and Illyrians""], ""ans..."
497,"Along with the desire for independence, tensio...",What position did Muslims have in the Indian p...,"{""text"": [""minority""], ""answer_start"": [147]}"
498,"From childhood, Gaddafi was aware of the invol...",What country occupied Libya during World War II?,"{""text"": [""his nation was occupied by Italy""],..."


## Temporal

In [42]:
t = crossproduct(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'There was a change in profession'
# test = MFT(**t, expect=expect_squad, capability='Temporal', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [43]:
dataset_fmt(t, "temproal")

,context,question,answers
0,"Both Bill and Amy were investors, but there wa...",Who is an actress?,"{""text"": [""Bill""], ""answer_start"": [5]}"
1,"Both Amy and Bill were investors, but there wa...",Who is an actress?,"{""text"": [""Bill""], ""answer_start"": [13]}"
2,"Both Larry and Jay were editors, but there was...",Who is a photographer?,"{""text"": [""Larry""], ""answer_start"": [5]}"
3,"Both Jay and Larry were editors, but there was...",Who is a photographer?,"{""text"": [""Larry""], ""answer_start"": [13]}"
4,"Both Alice and Larry were waitresss, but there...",Who is a photographer?,"{""text"": [""Alice""], ""answer_start"": [5]}"
...,...,...,...
967,"Both Rose and Judy were investors, but there w...",Who is a musician?,"{""text"": [""Judy""], ""answer_start"": [14]}"
968,"Both Rose and Greg were artists, but there was...",Who is a DJ?,"{""text"": [""Rose""], ""answer_start"": [5]}"
969,"Both Greg and Rose were artists, but there was...",Who is a DJ?,"{""text"": [""Rose""], ""answer_start"": [14]}"
970,"Both Colin and Dick were executives, but there...",Who is an editor?,"{""text"": [""Colin""], ""answer_start"": [5]}"


In [44]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ), 
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Understanding before / after -> first / last.'
# test = MFT(**t, expect=expect_squad, capability='Temporal', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)


In [45]:
dataset_fmt(t, "before_after")

,context,question,answers
0,Virginia became a reporter before Roy did.,Who became a reporter first?,"{""text"": [""Virginia""], ""answer_start"": [0]}"
1,Virginia became a reporter before Roy did.,Who became a reporter last?,"{""text"": [""Roy""], ""answer_start"": [34]}"
2,Roy became a reporter after Virginia did.,Who became a reporter first?,"{""text"": [""Virginia""], ""answer_start"": [28]}"
3,Roy became a reporter after Virginia did.,Who became a reporter last?,"{""text"": [""Roy""], ""answer_start"": [0]}"
4,Donald became a nurse before Scott did.,Who became a nurse first?,"{""text"": [""Donald""], ""answer_start"": [0]}"
...,...,...,...
1983,Stephen became a interpreter after Henry did.,Who became a interpreter last?,"{""text"": [""Stephen""], ""answer_start"": [0]}"
1984,Katherine became a entrepreneur before Jessica...,Who became a entrepreneur first?,"{""text"": [""Katherine""], ""answer_start"": [0]}"
1985,Katherine became a entrepreneur before Jessica...,Who became a entrepreneur last?,"{""text"": [""Jessica""], ""answer_start"": [39]}"
1986,Jessica became a entrepreneur after Katherine ...,Who became a entrepreneur first?,"{""text"": [""Katherine""], ""answer_start"": [36]}"


## Negation

In context

In [46]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Negation in context, may or may not be in question'
# test = MFT(**t, expect=expect_squad, capability='Negation', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [47]:
dataset_fmt(t, "negation")

,context,question,answers
0,Gary is not a reporter. Evelyn is.,Who is a reporter?,"{""text"": [""Evelyn""], ""answer_start"": [24]}"
1,Gary is not a reporter. Evelyn is.,Who is not a reporter?,"{""text"": [""Gary""], ""answer_start"": [0]}"
2,Evelyn is a reporter. Gary is not.,Who is a reporter?,"{""text"": [""Evelyn""], ""answer_start"": [0]}"
3,Evelyn is a reporter. Gary is not.,Who is not a reporter?,"{""text"": [""Gary""], ""answer_start"": [22]}"
4,Don is not an actor. Benjamin is.,Who is an actor?,"{""text"": [""Benjamin""], ""answer_start"": [21]}"
...,...,...,...
1983,Philip is a photographer. Alan is not.,Who is not a photographer?,"{""text"": [""Alan""], ""answer_start"": [26]}"
1984,Catherine is not a DJ. James is.,Who is a DJ?,"{""text"": [""James""], ""answer_start"": [23]}"
1985,Catherine is not a DJ. James is.,Who is not a DJ?,"{""text"": [""Catherine""], ""answer_start"": [0]}"
1986,James is a DJ. Catherine is not.,Who is a DJ?,"{""text"": [""James""], ""answer_start"": [0]}"


Not in context:

In [48]:

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Negation in question only.'
# test = MFT(**t, expect=expect_squad, capability='Negation', name=name, description='' )
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [49]:
dataset_fmt(t, "negation_question")

,context,question,answers
0,Virginia is an author. Lawrence is a photograp...,Who is an author?,"{""text"": [""Virginia""], ""answer_start"": [0]}"
1,Virginia is an author. Lawrence is a photograp...,Who is not an author?,"{""text"": [""Lawrence""], ""answer_start"": [23]}"
2,Virginia is an author. Lawrence is a photograp...,Who is a photographer?,"{""text"": [""Lawrence""], ""answer_start"": [23]}"
3,Virginia is an author. Lawrence is a photograp...,Who is not a photographer?,"{""text"": [""Virginia""], ""answer_start"": [0]}"
4,Lawrence is a photographer. Virginia is an aut...,Who is an author?,"{""text"": [""Virginia""], ""answer_start"": [28]}"
...,...,...,...
3811,Ruth is an interpreter. Joan is a waitress.,Who is not a waitress?,"{""text"": [""Ruth""], ""answer_start"": [0]}"
3812,Joan is a waitress. Ruth is an interpreter.,Who is an interpreter?,"{""text"": [""Ruth""], ""answer_start"": [20]}"
3813,Joan is a waitress. Ruth is an interpreter.,Who is not an interpreter?,"{""text"": [""Joan""], ""answer_start"": [0]}"
3814,Joan is a waitress. Ruth is an interpreter.,Who is a waitress?,"{""text"": [""Joan""], ""answer_start"": [0]}"


## Fairness spinoff

In [50]:
import collections
fewer_profs = ['doctor', 'nurse', 'secretary', 'CEO']
t = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.', 'Who is {a:prof}?', '{female}', 'woman', '{prof}'),
        ('{female} is not {a:prof}, {male} is.', 'Who is {a:prof}?', '{male}', 'man', '{prof}'),
    ],
#     prof=professions + ['doctor'],
    prof=fewer_profs,
    remove_duplicates=True,
    nsamples=1000,
    unroll=True,
    save=True,
    )
data = [(d[0], d[1]) for d in t.data]
labels = [d[2] for d in t.data]
meta = [(d[3], d[4]) for d in t.data]

# test = MFT(data, expect=expect_squad, labels=labels, meta=meta, templates=t.templates,
#           name='M/F failure rates should be similar for different professions', capability='Fairness',
#           description='Using negation in context.')
# test.run(predconfs, n=100)

# def print_fair(test):
#     c = collections.Counter(test.meta)
#     fail = collections.Counter([tuple(x) for x in np.array(test.meta)[test.fail_idxs()]])
#     profs = set()
#     for sex, prof in fail:
#         profs.add(prof)
#     prof_fail = {}
#     get_fail = lambda f:fail[f] / c[f]
#     for prof in profs:
#         fail_m = get_fail(('man', prof))
#         fail_f = get_fail(('woman', prof))
#         prof_fail[prof] = (fail_m, fail_f)
#     print('%-13s fail_men fail_women (count)' % 'profession')
#     for prof, vs in sorted(prof_fail.items(), key=lambda x:max(x[1][0], x[1][1]), reverse=True):
#         fail_m, fail_f = vs
#         print('%-13s   %.1f      %.1f     (%d)' % (prof, 100 * fail_m, 100 * fail_f, c[('man', prof)]))
# print_fair(test)
# suite.add(test)

In [60]:
t.data = [[i] for i in data]
t.labels = [[i] for i in labels]

In [63]:
dataset_fmt(t, "fair")

,context,question,answers
0,"Jay is not a CEO, Anne is.",Who is a CEO?,"{""text"": [""Anne""], ""answer_start"": [18]}"
1,"Anne is not a CEO, Jay is.",Who is a CEO?,"{""text"": [""Jay""], ""answer_start"": [19]}"
2,"Johnny is not a nurse, Heather is.",Who is a nurse?,"{""text"": [""Heather""], ""answer_start"": [23]}"
3,"Heather is not a nurse, Johnny is.",Who is a nurse?,"{""text"": [""Johnny""], ""answer_start"": [24]}"
4,"Henry is not a nurse, Ann is.",Who is a nurse?,"{""text"": [""Ann""], ""answer_start"": [22]}"
...,...,...,...
1977,"Barbara is not a doctor, Peter is.",Who is a doctor?,"{""text"": [""Peter""], ""answer_start"": [25]}"
1978,"Tim is not a CEO, Rose is.",Who is a CEO?,"{""text"": [""Rose""], ""answer_start"": [18]}"
1979,"Rose is not a CEO, Tim is.",Who is a CEO?,"{""text"": [""Tim""], ""answer_start"": [19]}"
1980,"Brian is not a doctor, Harriet is.",Who is a doctor?,"{""text"": [""Harriet""], ""answer_start"": [23]}"


## Coref

Basic coref

In [65]:
if 'actress' in professions:
    professions.remove('actress')

In [67]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ), 
            (
                'Who is {a:prof2}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Basic coref, he / she'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [68]:
dataset_fmt(t, "basic_coref")

,context,question,answers
0,"Jeff and Dorothy are friends. He is an actor, ...",Who is an actor?,"{""text"": [""Jeff""], ""answer_start"": [0]}"
1,"Jeff and Dorothy are friends. He is an actor, ...",Who is an interpreter?,"{""text"": [""Dorothy""], ""answer_start"": [9]}"
2,"Dorothy and Jeff are friends. He is an actor, ...",Who is an actor?,"{""text"": [""Jeff""], ""answer_start"": [12]}"
3,"Dorothy and Jeff are friends. He is an actor, ...",Who is an interpreter?,"{""text"": [""Dorothy""], ""answer_start"": [0]}"
4,Jeff and Dorothy are friends. She is an interp...,Who is an actor?,"{""text"": [""Jeff""], ""answer_start"": [0]}"
...,...,...,...
3899,"Katie and Brian are friends. He is a waitress,...",Who is an executive?,"{""text"": [""Katie""], ""answer_start"": [0]}"
3900,Brian and Katie are friends. She is an executi...,Who is a waitress?,"{""text"": [""Brian""], ""answer_start"": [0]}"
3901,Brian and Katie are friends. She is an executi...,Who is an executive?,"{""text"": [""Katie""], ""answer_start"": [10]}"
3902,Katie and Brian are friends. She is an executi...,Who is a waitress?,"{""text"": [""Brian""], ""answer_start"": [10]}"


In [69]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. His mom is {a:prof}.',
            '{female} and {male} are friends. His mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{male}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))
t += crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. Her mom is {a:prof}.',
            '{female} and {male} are friends. Her mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))

name = 'Basic coref, his / her'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [70]:
dataset_fmt(t, "basic_coref2")

,context,question,answers
0,Sam and Alison are friends. His mom is an actor.,Whose mom is an actor?,"{""text"": [""Sam""], ""answer_start"": [0]}"
1,Alison and Sam are friends. His mom is an actor.,Whose mom is an actor?,"{""text"": [""Sam""], ""answer_start"": [11]}"
2,Jay and Sara are friends. His mom is a writer.,Whose mom is a writer?,"{""text"": [""Jay""], ""answer_start"": [0]}"
3,Sara and Jay are friends. His mom is a writer.,Whose mom is a writer?,"{""text"": [""Jay""], ""answer_start"": [9]}"
4,Edward and Anna are friends. His mom is an arc...,Whose mom is an architect?,"{""text"": [""Edward""], ""answer_start"": [0]}"
...,...,...,...
993,Sally and Andrew are friends. Her mom is an in...,Whose mom is an interpreter?,"{""text"": [""Sally""], ""answer_start"": [0]}"
994,Johnny and Lauren are friends. Her mom is a pr...,Whose mom is a producer?,"{""text"": [""Lauren""], ""answer_start"": [11]}"
995,Lauren and Johnny are friends. Her mom is a pr...,Whose mom is a producer?,"{""text"": [""Lauren""], ""answer_start"": [0]}"
996,Stephen and Wendy are friends. Her mom is an a...,Whose mom is an attorney?,"{""text"": [""Wendy""], ""answer_start"": [12]}"


Former, latter

In [71]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Former / Latter'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [72]:
dataset_fmt(t, "former")

,context,question,answers
0,Kathy and Harriet are friends. The former is a...,Who is an artist?,"{""text"": [""Kathy""], ""answer_start"": [0]}"
1,Harriet and Kathy are friends. The latter is a...,Who is an artist?,"{""text"": [""Kathy""], ""answer_start"": [12]}"
2,Kathy and Harriet are friends. The former is a...,Who is an artist?,"{""text"": [""Kathy""], ""answer_start"": [0]}"
3,Harriet and Kathy are friends. The former is a...,Who is an artist?,"{""text"": [""Kathy""], ""answer_start"": [12]}"
4,Fred and Tim are friends. The former is an actor.,Who is an actor?,"{""text"": [""Fred""], ""answer_start"": [0]}"
...,...,...,...
1915,Leslie and Jay are friends. The former is an o...,Who is an escort?,"{""text"": [""Jay""], ""answer_start"": [11]}"
1916,Deborah and Alice are friends. The former is a...,Who is an organizer?,"{""text"": [""Deborah""], ""answer_start"": [0]}"
1917,Alice and Deborah are friends. The latter is a...,Who is an organizer?,"{""text"": [""Deborah""], ""answer_start"": [10]}"
1918,Deborah and Alice are friends. The former is a...,Who is an organizer?,"{""text"": [""Deborah""], ""answer_start"": [0]}"


## SRL

In [75]:
import pattern
import pattern.en
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('loves')[0]
b = pattern.en.tenses('stolen')[0]
pverb = [(pattern.en.conjugate(v, *a), pattern.en.conjugate(v, *b)) for v in pverb]

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}.',
        ],
        'qas': [
            (
                'Who {v[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {v[1]}?',
                '{first_name2}'
            ), 
        ]
        
    },
    v=pverb,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Agent / object distinction'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='SRL')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)

In [76]:
dataset_fmt(t, "agent")

,context,question,answers
0,Martin dislikes Jean.,Who dislikes?,"{""text"": [""Martin""], ""answer_start"": [0]}"
1,Martin dislikes Jean.,Who is disliked?,"{""text"": [""Jean""], ""answer_start"": [16]}"
2,Jean is disliked by Martin.,Who dislikes?,"{""text"": [""Martin""], ""answer_start"": [20]}"
3,Jean is disliked by Martin.,Who is disliked?,"{""text"": [""Jean""], ""answer_start"": [0]}"
4,Katherine trusts Frances.,Who trusts?,"{""text"": [""Katherine""], ""answer_start"": [0]}"
...,...,...,...
1991,Suzanne is believed by Kate.,Who is believed?,"{""text"": [""Suzanne""], ""answer_start"": [0]}"
1992,Dan attacks Greg.,Who attacks?,"{""text"": [""Dan""], ""answer_start"": [0]}"
1993,Dan attacks Greg.,Who is attacked?,"{""text"": [""Greg""], ""answer_start"": [12]}"
1994,Greg is attacked by Dan.,Who attacks?,"{""text"": [""Dan""], ""answer_start"": [20]}"


In [77]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}. {first_name2} {v[0]} {first_name3}.',
            '{first_name} {v[0]} {first_name2}. {first_name3} is {v[1]} by {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name2} {v[0]} {first_name3}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name3} is {v[1]} by {first_name2}.',
        ],
        'qas': [
            (
                'Who {v[0]} {first_name2}?',
                '{first_name}'
            ), 
            (
                'Who {v[0]} {first_name3}?',
                '{first_name2}'
            ), 
            (
                'Who is {v[1]} by {first_name}?',
                '{first_name2}'
            ), 
            (
                'Who is {v[1]} by {first_name2}?',
                '{first_name3}'
            ), 
        ]
        
    },
    save=True,
    v=pverb,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Agent / object distinction with 3 agents'
# test = MFT(**t, expect=expect_squad, name=name, description='', capability='SRL')
# test.run(predconfs, n=100)
# test.summary(n=3, format_example_fn=format_squad_with_context)
# suite.add(test)


In [78]:
dataset_fmt(t, "agent2")

,context,question,answers
0,Victoria notices Fiona. Fiona notices Laura.,Who notices Fiona?,"{""text"": [""Victoria""], ""answer_start"": [0]}"
1,Victoria notices Fiona. Fiona notices Laura.,Who notices Laura?,"{""text"": [""Fiona""], ""answer_start"": [17]}"
2,Victoria notices Fiona. Fiona notices Laura.,Who is noticed by Victoria?,"{""text"": [""Fiona""], ""answer_start"": [17]}"
3,Victoria notices Fiona. Fiona notices Laura.,Who is noticed by Fiona?,"{""text"": [""Laura""], ""answer_start"": [38]}"
4,Victoria notices Fiona. Laura is noticed by Fi...,Who notices Fiona?,"{""text"": [""Victoria""], ""answer_start"": [0]}"
...,...,...,...
7899,Nick is disliked by Catherine. Nick dislikes S...,Who is disliked by Nick?,"{""text"": [""Sara""], ""answer_start"": [45]}"
7900,Nick is disliked by Catherine. Sara is dislike...,Who dislikes Nick?,"{""text"": [""Catherine""], ""answer_start"": [20]}"
7901,Nick is disliked by Catherine. Sara is dislike...,Who dislikes Sara?,"{""text"": [""Nick""], ""answer_start"": [0]}"
7902,Nick is disliked by Catherine. Sara is dislike...,Who is disliked by Catherine?,"{""text"": [""Nick""], ""answer_start"": [0]}"


In [86]:
import pandas as pd
import glob

path = 'new_data' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [87]:
len(frame)

59350

In [86]:
frame.to_csv('checklist_train.csv', index=None)

In [111]:
combined = pd.concat([
    df_train[['context','question','answers']],
    frame]
).iloc[np.random.permutation(len(combined))].reset_index(drop=True)

In [112]:
combined.to_csv('combined_train.csv', index=None)

In [113]:
dataset = datasets.load_dataset('csv', data_files='combined_train.csv')
def format_dataset(example):
    """
    string to dictionary 
    dictionary became string when saved to csv. need to convert it back to dict.  
    """
    example['answers'] = json.loads(example['answers']) 
    return example
dataset = dataset.map(format_dataset)

Using custom data configuration default-1d3a3554611ef5f9


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/syang/.cache/huggingface/datasets/csv/default-1d3a3554611ef5f9/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/146949 [00:00<?, ?ex/s]